In [ ]:

%pip install -U openai jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
!rm -rf dspy
!git clone https://github.com/stanfordnlp/dspy.git
%cd dspy
!python setup.py install

Cloning into 'dspy'...
remote: Enumerating objects: 26886, done.
remote: Counting objects: 100% (3865/3865), done.
remote: Compressing objects: 100% (1244/1244), done.
remote: Total 26886 (delta 2946), reused 3119 (delta 2589), pack-reused 23021
Receiving objects: 100% (26886/26886), 26.09 MiB | 14.55 MiB/s, done.
Resolving deltas: 100% (11471/11471), done.
/content/dspy
/usr/local/lib/python3.10/dist-packages/setuptools/config/_apply_pyprojecttoml.py:69: SetuptoolsWarning: `install_requires` overwritten in `pyproject.toml` (dependencies)
  corresp(dist, value, root_dir)
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.

In [ ]:
!pip install backoff

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Constructing Base Pipeline

In [ ]:
import dspy

from dspy.evaluate.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, BayesianSignatureOptimizer


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

In [ ]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=1000)
dspy.configure(lm=turbo)

In [ ]:
class StudentFeedback(dspy.Signature):
    """A student is learning English. You are assessing a spoken utterance. In at most two sentences, summarize (1) their specific strengths in English skills and (2) things they can work on to improve. Address the student in the second person. Include specific examples that the student can learn from. Be colloquial, as if in spoken conversation."""

    convo = dspy.InputField()
    output = dspy.OutputField(desc="Treat this as a spoken conversation, so be succinct, colloquial, and empathetic.")

In [ ]:
class OfferFeedback(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_feedback = dspy.ChainOfThought(StudentFeedback)

    def forward(self, convo):
        answer = self.generate_feedback(convo=convo)
        return answer

In [ ]:
zeroshot = OfferFeedback()

In [ ]:
convo = """- Yeah, I always talk to my parents about the student.
- Yeah, so the person that I met, you know, that I've met, that I want to study with is one of my classmates, Tom, and how I met this, met Tom is when I was in high school and he's in my next classroom and he is one of the best students in the other class and I really admire his intelligence and I learn a lot from him and I've known him since, you know, 2016 and he is one of the best students in the other class so that's why I want to study with him.
"""

zeroshot.forward(convo)

Prediction(
    rationale='assess your spoken utterance. We can see that you have a strong grasp of English vocabulary and grammar, as evidenced by your ability to express complex ideas and use a variety of sentence structures. However, you may want to work on your pronunciation and intonation, as some of your words were not clear and your speech lacked natural rhythm. For example, you said "met" instead of "meet" and "classroom" instead of "classmate." Practicing with a native speaker or using online resources can help you improve in this area. Keep up the good work!',
    output='You have a great understanding of English vocabulary and grammar, which is evident in your ability to express complex ideas and use different sentence structures. However, you may want to work on your pronunciation and intonation, as some of your words were not clear and your speech lacked natural rhythm. For example, you said "met" instead of "meet" and "classroom" instead of "classmate." Practicing with a 

In [ ]:
zeroshot.forward(convo)





A student is learning English. You are assessing a spoken utterance. In at most two sentences, summarize (1) their specific strengths in English skills and (2) things they can work on to improve. Address the student in the second person. Include specific examples that the student can learn from. Be colloquial, as if in spoken conversation.

---

Follow the following format.

Convo: ${convo}
Reasoning: Let's think step by step in order to ${produce the output}. We ...
Output: Treat this as a spoken conversation, so be succinct, colloquial, and empathetic.

---

Convo: - Yeah, I always talk to my parents about the student. - Yeah, so the person that I met, you know, that I've met, that I want to study with is one of my classmates, Tom, and how I met this, met Tom is when I was in high school and he's in my next classroom and he is one of the best students in the other class and I really admire his intelligence and I learn a lot from him and I've known him since, you know, 2016 and he

In [ ]:
import dspy

# Define the signature for autoamtic assessments.
class Assess(dspy.Signature):
    """Assess the quality of a spoken utterance along the specified dimension."""

    context = dspy.InputField(desc='ignore if N/A')
    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="Yes or No")


gpt4T = dspy.OpenAI(model='gpt-4-turbo-preview', max_tokens=1000, model_type='chat')
METRIC = None

def metric(gold, pred, trace=None):
    convo, utt = gold.convo, pred.output


    # understandable = "Is the spoken utterance simple enough to beginner English learners?"

    empathetic = "Does the assessed spoken utterance sound empathetic and encouraging?"
    example_include = "Does the spoken utterance include a specific example for the student to learn from?"
    colloquial = "Would someone say this utterance in a beginner English classroom conversation?"
    correct = f"The student said the following utterances:\n{convo}\nDoes the assessed spoken utterance correctly point out the specific issues and strengths in the student's utterances from a language skill perspective?"

    with dspy.context(lm=gpt4T):
        empathetic = dspy.Predict(Assess)(context='N/A', assessed_text=utt, assessment_question=empathetic)
        example_include = dspy.Predict(Assess)(context='N/A', assessed_text=utt, assessment_question=example_include)
        # understandable = dspy.Predict(Assess)(context='N/A', assessed_text=utt, assessment_question=understandable)
        colloquial =  dspy.Predict(Assess)(context='N/A', assessed_text=utt, assessment_question=colloquial)
        correct =  dspy.Predict(Assess)(context='N/A', assessed_text=utt, assessment_question=correct)

    empathetic, example_include, colloquial, correct = [m.assessment_answer.split()[0].lower() == 'yes' for m in [empathetic, example_include, colloquial, correct]]
    score = (empathetic + example_include + 2 * colloquial + correct) if correct and empathetic and colloquial and len(utt) < 500 else 0

    # if METRIC is not None:
    #     if METRIC == 'correct': return correct
    #     if METRIC == 'engaging': return engaging
    #     if METRIC == 'faithful': return faithful

    if trace is not None: return score >= 5
    return score / 5

In [ ]:
import json
from dspy import Example

convos = json.load(open("<PATH_TO_UTTERANCES>"))
trainset, valset, devset = convos["train"], convos["val"], convos["test"]

trainset = trainset[:125]
trainset = [Example({"convo": x}).with_inputs("convo") for x in trainset]
valset = valset[:200]
valset = [Example({"convo": x}).with_inputs("convo") for x in valset]
devset = devset[:300]
devset = [Example({"convo": x}).with_inputs("convo") for x in devset]

In [ ]:
evaluate = Evaluate(metric=metric, devset=valset, num_threads=8, display_progress=True, display_table=5)
evaluate(zeroshot)

In [ ]:
# Set up the optimizer. We'll use very minimal hyperparameters for this example.
# Just do random search with ~3 attempts, and in each attempt, bootstrap <= 3 traces.
optimizer = BayesianSignatureOptimizer(metric=metric, prompt_model=gpt4T, task_model=turbo)


optimized_chain = optimizer.compile(zeroshot, devset=trainset, optuna_trials_num=20, max_bootstrapped_demos=5, max_labeled_demos=5, eval_kwargs=dict(num_threads=8, display_progress=True))

  6%|▋         | 8/125 [00:00<00:00, 162.26it/s]


Bootstrapped 5 full traces after 9 examples in round 0.


 10%|█         | 13/125 [00:00<00:00, 112.67it/s]


Bootstrapped 5 full traces after 14 examples in round 0.


  6%|▌         | 7/125 [00:00<00:01, 92.58it/s]


...

Average Metric: 53.0 / 100  (53.0%)


Average Metric: 75.0 / 100  (75.0): 100%|██████████| 100/100 [00:00<00:00, 116.95it/s]
[I 2024-02-27 20:10:28,793] Trial 18 pruned. 


Average Metric: 75.0 / 100  (75.0%)


Average Metric: 53.0 / 100  (53.0): 100%|██████████| 100/100 [00:00<00:00, 102.67it/s]
[I 2024-02-27 20:10:29,910] Trial 19 pruned. 


Average Metric: 53.0 / 100  (53.0%)


In [ ]:
optimized_chain.save("emp_bot.json")

In [ ]:
evaluate(optimized_chain)

In [ ]:
new_eval = Evaluate(metric=metric, devset=devset, num_threads=8, display_progress=True, display_table=5)

In [ ]:
new_eval(optimized_chain)

Average Metric: 268.0 / 300  (89.3): 100%|██████████| 300/300 [02:51<00:00,  1.75it/s]

Average Metric: 268.0 / 300  (89.3%)


,convo,rationale,output,metric
0,"- Having break during work of studies essential helps to maintain focus, reduce stress and improve productivity. - Yes, there are more activities for young...","assess your spoken English skills. You have a good understanding of the topic and can express your thoughts clearly. However, you can work on your...","You have a good understanding of the topic and can express your thoughts clearly, which is great! However, you can work on your grammar and...",1.0
1,"- Why do you think so? - In your view, should the media only publish good news? - How does social media help people access...","assess your spoken English skills. You have a good understanding of the topic and can express your thoughts clearly. However, you can work on your...","You have a good understanding of the topic and can express your thoughts clearly, which is great! However, you can work on your grammar and...",1.0
2,"- Well, that was several years ago. I think when I graduated from college, I had a chance at the time to went back to...","assess your spoken English skills. You have a good understanding of the topic and can express yourself clearly. However, you can work on your verb...","You have a good understanding of the topic and can express yourself clearly, which is great! However, you can work on your verb tenses and...",1.0
3,"- Oh yes, I have learned how to sing before. I've taken some classes and practiced on my own to improve my techniques in singing....","assess your spoken English skills. You have a good understanding of the topic and can express your thoughts clearly. However, you can work on your...","You have a good understanding of the topic and can express your thoughts clearly, which is great! However, you can work on your grammar and...",1.0
4,"- There are many TV programs about wild animals in my country. Some of the most popular shows focus on wildlife conservation, while others take...","assess your spoken English skills. You have a good understanding of the topic and can express yourself clearly. However, you can work on your grammar...","You have a good understanding of the topic and can express yourself clearly, which is great! However, you can work on your grammar and sentence...",1.0


89.33

In [ ]:
new_eval(zeroshot)

Average Metric: 178.40000000000015 / 300  (59.5): 100%|██████████| 300/300 [01:34<00:00,  3.19it/s]

Average Metric: 178.40000000000015 / 300  (59.5%)


,convo,rationale,output,metric
0,"- Having break during work of studies essential helps to maintain focus, reduce stress and improve productivity. - Yes, there are more activities for young...","assess your spoken utterance. We can see that you have a strong understanding of the English language, as you are able to express your thoughts...","You have a great grasp of the English language, as you are able to express your thoughts and ideas clearly and coherently. However, you can...",0.0
1,"- Why do you think so? - In your view, should the media only publish good news? - How does social media help people access...","assess your spoken utterance. We can see that you have a strong understanding of how to ask questions and engage in a conversation. However, you...","You're doing great with asking questions and keeping the conversation going! However, you can work on your pronunciation and enunciation to improve your clarity and...",1.0
2,"- Well, that was several years ago. I think when I graduated from college, I had a chance at the time to went back to...","assess your spoken utterance. We can see that you have a strong grasp of verb tenses and use them correctly in your sentences. However, you...","You have a good understanding of verb tenses, but you can work on your pronunciation and intonation to make your speech more natural. For example,...",1.0
3,"- Oh yes, I have learned how to sing before. I've taken some classes and practiced on my own to improve my techniques in singing....",assess your spoken utterance. We can see that you have a strong understanding of musical elements and how they work together to create a beautiful...,"You have a strong understanding of musical elements and the power of singing to bring people together. However, you mentioned that you get tense when...",0.0
4,"- There are many TV programs about wild animals in my country. Some of the most popular shows focus on wildlife conservation, while others take...","assess your spoken English skills. We can see that you have a strong grasp of vocabulary and can express yourself clearly and confidently. However, you...","You have a great vocabulary and can express yourself confidently, but you can work on your fluency and pronunciation to make your speech sound more...",0.8


59.47

In [ ]:
turbo.inspect_history()





Proposed Instruction: You're playing the role of an encouraging English tutor for a student who is actively learning and practicing their English through conversation. Your task is to listen attentively to their spoken utterances and provide constructive feedback. In your response, kindly highlight (1) one specific strength they showed or an aspect they did well in during the conversation, complimenting their effort or skill in English, and (2) offer one focused suggestion on how they can improve further, making it actionable and clear. Use colloquial language to maintain the conversational tone, directly addressing the student with "you", and where possible, reference specific examples from their speech to illustrate your points. Your feedback should feel like a supportive nudge towards their language learning journey, keeping it concise and personalized.

---

Follow the following format.

Convo: ${convo}
Reasoning: Let's think step by step in order to ${produce the output}. We .

In [ ]:
reload_chain = OfferFeedback()
reload_chain.load("emp_bot.json")

In [ ]:
evaluate(reload_chain)

In [ ]:
for x in valset[:20]:
  print(x)
  print(reload_chain(x["convo"]))

In [ ]:
reload_chain.forward(convo)

Prediction(
    rationale='assess your spoken English skills. You have a good understanding of grammar and vocabulary, and you are able to express your thoughts clearly. However, you can work on your sentence structure and pronunciation to make your speech more fluent and natural. For example, instead of saying "the student", you can say "my classmate". Also, try to enunciate each word clearly and vary your tone to make your speech more engaging. Keep practicing and you will continue to improve!',
    output='You have a good understanding of grammar and vocabulary, and you are able to express your thoughts clearly. However, you can work on your sentence structure and pronunciation to make your speech more fluent and natural. For example, instead of saying "the student", you can say "my classmate". Also, try to enunciate each word clearly and vary your tone to make your speech more engaging. Keep practicing and you will continue to improve!'
)